In [26]:
import spacy
import re 

In [27]:
def extraer_metadatos_gaceta_spacy(texts_pdf):
    """
    Extrae metadatos de una gaceta del Congreso usando spaCy y expresiones regulares.
    """
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(texts_pdf)
    
    metadatos = {
        "fecha": None,
        "directores": None,
        "num_paginas": None,
        "anio_romano": None,
        "anio": None,
        "nombre": "Gaceta del Congreso",
        "subtitulo": None,
        "issn": None,
        "rama": "Rama Legislativa del Poder Público",
        "camara": "SENADO DE LA REPÚBLICA",
        "tipo_documento": "CONCEPTOS"
    }
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            metadatos["fecha"] = ent.text
        elif ent.label_ == "CARDINAL" and "páginas" in ent.sent.text.lower():
            metadatos["num_paginas"] = int(ent.text)
        elif ent.label_ == "ORG" and "directores" in ent.sent.text.lower():
            metadatos["directores"] = ent.text
        elif ent.label_ == "MISC" and "ISSN" in ent.sent.text:
            metadatos["issn"] = ent.text.split()[-1]

    # --- 1. Fecha ---
    # spaCy es bueno reconociendo fechas, pero el formato específico necesita regex.
    pattern_date = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
    match_date = re.search(pattern_date, texts_pdf)  # ¡Usamos regex!
    if match_date:
        meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
                 "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
        day_month = int(match_date.group(2))
        month_nu, = meses[match_date.group(3).lower()]
        anio = int(match_date.group(4))
        metadatos["fecha"] = f"{day_month:02d}/{month_nu:02d}/{anio}"


        # --- 2. Directores ---
    # Combinamos spaCy (NER para personas) con regex para el contexto.
    directors = []
    for ent in doc.ents:
        if ent.label_ == "PER":  # Entidad de tipo "Persona"
            # Busca la línea que contiene la entidad y verifica si está cerca de "DIRECTORES"
            linea = ent.sent.text  # Obtiene la oración completa
            if "DIRECTORES" in linea.upper():
                # Podríamos usar una regex aquí también para más precisión.
                directors.append(ent.text)
    metadatos["directores"] = ", ".join(directors)  


    # --- 3. Edición de n páginas --- (Regex)
    patterns_pages = r"(?i)EDICI[ÓO]N\s+DE\s+(\d+)\s+P[ÁA]GINAS"
    match_pages = re.search(patterns_pages, texts_pdf)
    if match_pages:
        metadatos["num_paginas"] = int(match_pages.group(1))

    # --- 4. Año (Números Romanos) --- (Regex)
    pattern_anio = r"AÑO\s+(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))"
    match_anio = re.search(pattern_anio, texts_pdf)
    if match_anio:
        romano = match_anio.group(1)
        metadatos["anio_romano"] = romano
        metadatos["anio"] = romano_a_entero(romano)  # Función de conversión

    # --- 5. Nombre Principal ---
    pattern_name = r"(?i)GACETA\s+DEL\s+CONGRESO"
    if re.search(pattern_name, texts_pdf):
        metadatos["nombre"] = "Gaceta del Congreso"

    # --- 6. Subtítulo ---
    # Regex es más confiable para la estructura, spaCy podría fallar aquí
    pattern_subtitulo = r"(?i)GACETA\s+DEL\s+CONGRESO\s*\n*(.*?)\s*(?:AÑO|DIRECTORES)"
    match_subtitulo = re.search(pattern_subtitulo, texts_pdf)
    if match_subtitulo:
        metadatos["subtitulo"] = match_subtitulo.group(1).strip()
        
    # --- 7. Artículo, Ley ---
    pattern_articule =  r"(?i)ART[ÍI]CULO\s+\d+(?:\s*(?:[º°ª])|\s*[Nn][°º])[\s.,:;]*LEY\s+\d+(?:\s*(?:[º°ª])|\s*[Nn][°º])[\s.,:;]*DE[\s.,:;]*\d+"
    if re.search(pattern_articule, texts_pdf):
        metadatos["articulo_ley"] = True

    # --- 8. ISSN --- (Regex, formato muy específico)
    pattern_issn = r"(?i)ISSN\s+(\d{4}-\d{3}[\dX])"
    match_issn = re.search(pattern_issn, texts_pdf)
    if match_issn:
        metadatos["issn"] = match_issn.group(1)

    # --- 9. Rama Legislativa --- (Regex)
    pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+P[ÚU]BLICO"
    if re.search(pattern_branch, texts_pdf):
        metadatos["rama"] = "Rama Legislativa del Poder Público"

    # --- 10. Cámara o Senado ---
    # Podríamos usar spaCy para ORG, pero regex es más simple y preciso en este caso
    pattern_entity = r"(?i)(C[ÁA]MARA\s+DE\s+REPRESENTANTES|SENADO\s+DE\s+LA\s+REP[ÚU]BLICA)"
    match_entity = re.search(pattern_entity, texts_pdf, re.IGNORECASE)
    if match_entity:
            metadatos["camara"] = match_entity.group(1)

   # --- 11. Tipo de Documento ---
    pattern_type = r"^(Ponencia|Acta|Proyecto\s+de\s+Ley|Informe|Resolución|Concepto|Proposición|Constancia|Objeciones|Textos|Pliego)(.*)"
    for sent in doc.sents:  # Itera sobre las *oraciones* de spaCy
        match_type = re.search(pattern_type, sent.text, re.IGNORECASE) #se busca la coincidencia por texto
        if match_type:
            metadatos["tipo_documento"] = match_type.group(1).strip()
            metadatos["descripcion"] = match_type.group(2).strip()  # Descripción
            break  # Importante: Se detiene después de la primera coincidencia

    return metadatos


In [28]:

def romano_a_entero(romano):
    """Convierte un número romano a entero (MISMA FUNCIÓN)."""
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

In [29]:
def extraer_metadatos_gaceta_spacy(texto):
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(texto)
    
    metadatos = {
        "fecha": None,
        "directores": None,
        "num_paginas": None,
        "anio_romano": None,
        "anio": None,
        "nombre": "Gaceta del Congreso",
        "subtitulo": None,
        "issn": None,
        "rama": "Rama Legislativa del Poder Público",
        "camara": "SENADO DE LA REPÚBLICA",
        "tipo_documento": "CONCEPTOS"
    }
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            metadatos["fecha"] = ent.text
        elif ent.label_ == "CARDINAL" and "páginas" in ent.sent.text.lower():
            metadatos["num_paginas"] = int(ent.text)
        elif ent.label_ == "ORG" and "directores" in ent.sent.text.lower():
            metadatos["directores"] = ent.text
        elif ent.label_ == "MISC" and "ISSN" in ent.sent.text:
            metadatos["issn"] = ent.text.split()[-1]
    
    return metadatos
    

In [30]:
texto_prueba = """
REPÚBLICA DE COLOMBIA
CONGRESO
DE LA REPÚBLICA
DE COLOMBIA
SENADO DE LA REPÚBLICA
GACETA DEL CONGRESO
AÑO XXXII - N° 17
DIRECTORES:
SENADO Y CÁMARA
GREGORIO ELJACH PACHECO
JAIME LUIS LACOUTURE PEÑALOZA
(Artículo 36, Ley 5ª de 1992)
IMPRENTA NACIONAL DE COLOMBIA
www.imprenta.gov.co
Bogotá, D. C., jueves, 9 de febrero de 2023
SECRETARIO GENERAL DEL SENADO
www.secretariasenado.gov.co
ISSN 0123 - 9066
EDICIÓN DE 22 PÁGINAS

RAMA LEGISLATIVA DEL PODER PÚBLICO
SENADO DE LA REPÚBLICA
CONCEPTOS JURÍDICOS
CONCEPTO JURÍDICO DEL MINISTERIO DE SALUD Y PROTECCIÓN SOCIAL SOBRE...
"""

metadatos = extraer_metadatos_gaceta_spacy(texto_prueba)
print(metadatos)

{'fecha': None, 'directores': 'REPÚBLICA', 'num_paginas': None, 'anio_romano': None, 'anio': None, 'nombre': 'Gaceta del Congreso', 'subtitulo': None, 'issn': 'IMPRENTA', 'rama': 'Rama Legislativa del Poder Público', 'camara': 'SENADO DE LA REPÚBLICA', 'tipo_documento': 'CONCEPTOS'}
